In [10]:
import pandas as pd
import numpy as np
import os

Cleaning the Movies dataset by using external sources
---
A problem faced when working on the combination of the Movies and IMDB datasets was the different encoding of language 
and country in the respective attributes. In the IMDB dataset, countries and languages are written in their complete 
form, while the ISO codes are used in the Movielens dataset. 

To improve the performance of the matching algorithm it is possible to perform a preprocessing step that employs 
external ground truth to have a common dictionary between the two datasets. 


In [45]:
# Reading the required starting datasets
df1 = pd.read_csv('../pipeline/experiments/imdb_numeric.csv')
df2 = pd.read_csv('../pipeline/experiments/movies_reduced.csv')
df1 = df1.fillna('')
df2 = df2.fillna('')
# Removing capitalization and spaces
for col in df1.columns:
    if df1[col].dtype == 'object':
        df1[col] = df1[col].str.replace(' ', '_').str.strip().str.lower()
        df1[col] = df1[col].replace('ukn', '')
        df1[col] = df1[col].replace(np.nan, '')

for col in df2.columns:
    if df2[col].dtype == 'object':
        df2[col] = df2[col].str.replace(' ', '_').str.strip().str.lower()
        df2[col] = df2[col].replace('ukn', '')
        df2[col] = df2[col].replace(np.nan, '')


##### Replacing  countries with country codes
The mapping between country codes and country names was taken from open online sources and cleaned up to remove 
capitalization and spaces (this was a necessary step because of how the ER algorithm works). 

In [46]:
# mapping = {}
with open('../pipeline/experiments/country_codes.txt', 'r') as fp:
    lines = [_.strip().split(',') for idx, _ in enumerate(fp) if idx > 0]
    mapping = {line[0]:line[1] for line in lines}

for country in mapping:
    print('{} : {}'.format(country, mapping[country]))

afghanistan : af
akrotiri : ax
albania : al
algeria : ag
american_samoa : aq
andorra : an
angola : ao
anguilla : av
antarctica : ay
antigua_and_barbuda : ac
argentina : ar
armenia : am
aruba : aa
ashmore_and_cartier_islands : at
australia : as
austria : au
azerbaijan : aj
"bahamas : _the"
bahrain : ba
baker_island : fq
bangladesh : bg
barbados : bb
bassas_da_india : bs
belarus : bo
belgium : be
belize : bh
benin : bn
bermuda : bd
bhutan : bt
bolivia : bl
bosnia_and_herzegovina : bk
botswana : bc
bouvet_island : bv
brazil : br
british_indian_ocean_territory : io
british_virgin_islands : vi
brunei : bx
bulgaria : bu
burkina_faso : uv
burma : bm
burundi : by
cabo_verde : cv
cambodia : cb
cameroon : cm
canada : ca
cayman_islands : cj
central_african_republic : ct
chad : cd
chile : ci
china : ch
christmas_island : kt
clipperton_island : ip
cocos_(keeling)_islands : ck
colombia : co
comoros : cn
"congo : _republic_of_the"
cook_islands : cw
coral_sea_islands : cr
costa_rica : cs
cote_d'ivoire

In [47]:
# First replacement to clean most of the values.
val = df1[['country']].replace(np.nan, '').replace(list(mapping.keys()), list(mapping.values()))
# Some countries were not replaced correctly, so I am looking for them among the unique values.
[_ for _ in np.unique(val) if len(_) > 2]

['bahamas',
 'new_line',
 'official_site',
 'south_korea',
 'soviet_union',
 'usa',
 'west_germany']

In [48]:
# Missing values are manually replaced
df1.loc[df1['country'] == 'official_site', 'country'] = 'united_states'
df1.loc[df1['country'] == 'usa', 'country'] = 'united_states'
df1.loc[df1['country'] == 'west_germany', 'country'] = 'germany'
df1.loc[df1['country'] == 'new_line', 'country'] = ''
df1.loc[df1['country'] == 'bahamas', 'country'] = 'bf'
df1.loc[df1['country'] == 'south_korea', 'country'] = 'ks'
df1.loc[df1['country'] == 'soviet_union', 'country'] = 'rs'
df1.loc[df1['country'] == 'usa', 'country'] = 'us'

df1[['country']] = df1[['country']].replace(np.nan, '').replace(list(mapping.keys()), list(mapping.values()))
df2[['production_countries']] = df2[['production_countries']].replace(np.nan, '').replace(list(mapping.keys()), list(mapping.values()))


In [49]:
# Checking if all values have been replaced
print([_ for _ in np.unique(df1[['country']]) if len(_) > 2])
print([_ for _ in np.unique(df2[['production_countries']]) if len(_) > 2])

[]
[]


##### Replacing languages with language codes
Like in the case described above, language codes have been taken from online open sources. The same sequence of 
operations is performed again for the language attribute in both datasets. 

In [54]:
with open('../pipeline/experiments/language_codes.txt', 'r') as fp:
    lines = [_.strip().split(',') for _ in fp.readlines()]
    mapping = {line[0]:line[1] for line in lines}


In [62]:
df1['language'] = df1[['language']].replace(np.nan, '').replace(list(mapping.values()), list(mapping.keys()))

print([_ for _ in np.unique(df1[['language']]) if len(_) > 2])
print([_ for _ in pd.unique(df2['original_language']) if len(_) > 2])

['aboriginal', 'aramaic', 'dari', 'filipino', 'mandarin', 'maya', 'none']
[]


In [38]:
df2['original_language'].dtype

dtype('O')

In [64]:
df1.loc[df1['language'] == 'aboriginal', 'language'] = 'english'
df1.loc[df1['language'] == 'aramaic', 'language'] = 'english'
df1.loc[df1['language'] == 'filipino', 'language'] = 'english'
df1.loc[df1['language'] == 'maya', 'language'] = 'english'
df1.loc[df1['language'] == 'none', 'language'] = 'english'
df1.loc[df1['language'] == 'dari', 'language'] = 'english'
df1.loc[df1['language'] == 'osama', 'language'] = 'pushto'
df1.loc[df1['language'] == 'mandarin', 'language'] = 'chinese'
df1[['language']] = df1[['language']].replace(np.nan, '').replace(list(mapping.values()), list(mapping.keys()))

### Generating the concatenated dataset
The new, combined dataset is built by concatenating the two datasets. To do so, column names are uniformed to align them
when concatenating to prepare the dataset used in the ER task. 

In [ ]:
df2[['original_language']] = df2[['original_language']].replace(np.nan, '').replace(list(mapping.values()), list(mapping.keys()))
df1.rename(columns={
    'movie_title': 'title', 
    'director_name': 'director', 
    'actor_1_name':'actor_1', 
    'actor_3_name':'actor_3', 
    'actor_2_name':'actor_2', 
    'language': 'original_language', 
    'country':'production_countries', 
    'title_year': 'year', 
    'imdb_score':'vote_average'}, inplace=True)


In [70]:
concat = pd.concat([df1, df2])
concat.rename(columns={
    'original_language': 'language',
    'production_countries': 'country'
}, inplace=True)


/home/spoutnik23/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [74]:
concat.to_csv('../pipeline/experiments/movies-complete-preprocessed.csv', index=False)

To prepare the dataset used in the Schema Matching task, columns are renamed to avoid having any overlap in the 
concatenation phase.

In [75]:
df1.columns = ['imdb_' + str(_) for _ in range(len(df1.columns))]
df2.columns = ['movielens_' + str(_) for _ in range(len(df2.columns))]

In [ ]:
concat = pd.concat([df1, df2], ignore_index=True)
concat.to_csv('../pipeline/experiments/movies-complete-preprocessed-schema-matching.csv', index=False)
